# EnKF for Stock Price Estimation

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

from src.lorenz_systems.enkf import EnsembleKalmanFilter

In [2]:
# Fetch historical data for a stock
ticker = 'AAPL'
full_data = yf.download(ticker, start='2020-01-01', end='2023-03-01')

# Split data into assimilation and validation periods
assimilation_end_date = '2023-01-01'
prices = full_data.loc[:assimilation_end_date]['Adj Close'].values
validation_prices = full_data.loc[assimilation_end_date:]['Adj Close'].values
validation_dates = full_data.loc[assimilation_end_date:].index

C:\Users\marin\AppData\Local\Temp\ipykernel_22748\704229597.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  full_data = yf.download(ticker, start='2020-01-01', end='2023-03-01')
[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'

In [ ]:
# Define a simple random walk model for the EnKF
def random_walk_model(x, t_span, t_eval, process_noise_std):
    # This is a mock solution object to match the expected input for EnKF
    class MockSol:
        def __init__(self, y):
            self.y = y

    # Propagate the state with a random step
    y_end = x + np.random.normal(0, process_noise_std, x.shape)
    return MockSol(np.array([y_end]).T)

In [ ]:
# EnKF parameters
N_ens = 50
R = np.array([[0.1]])  # Observation noise
process_noise_std = 0.5

# Create a model function compatible with the EnKF class
model_func = lambda x, t_span, t_eval: random_walk_model(x, t_span, t_eval, process_noise_std)

enkf = EnsembleKalmanFilter(model=model_func, R=R, N=N_ens)

# Initial ensemble
X0 = np.random.normal(prices[0], 1.0, (1, N_ens))

# Run the EnKF
X_hist = [X0]
x_mean_hist = [np.mean(X0)]

for i in range(1, len(prices)):
    # Forecast
    X_f = enkf.forecast(X_hist[-1], t_span=None, t_eval=None)
    
    # Analysis
    X_a = enkf.analysis(X_f, np.array([prices[i]]), H=np.eye(1))
    
    X_hist.append(X_a)
    x_mean_hist.append(np.mean(X_a))

x_mean_hist = np.array(x_mean_hist)

In [ ]:
# Perform forecasts for different horizons
forecast_horizons = [1, 5, 10, 20, 50]
forecasts = {}

for horizon in forecast_horizons:
    X_forecast = X_hist[-1]
    current_forecasts = []
    for _ in range(horizon):
        X_forecast = enkf.forecast(X_forecast, t_span=None, t_eval=None)
        current_forecasts.append(np.mean(X_forecast))
    forecasts[horizon] = np.array(current_forecasts)

In [ ]:
# Calculate RMSE for each forecast horizon
rmse_scores = {}

for horizon, forecast_values in forecasts.items():
    # Ensure we don't go beyond the available validation data
    if horizon < len(validation_prices):
        true_values = validation_prices[1:horizon+1] # +1 because validation_prices[0] is the last assimilation day
        rmse = np.sqrt(np.mean((forecast_values - true_values)**2))
        rmse_scores[horizon] = rmse

In [ ]:
# Plot RMSE vs. Forecast Horizon
plt.figure(figsize=(10, 6))
plt.plot(list(rmse_scores.keys()), list(rmse_scores.values()), 'bo-')
plt.xlabel('Forecast Horizon (Days)')
plt.ylabel('RMSE')
plt.title('Forecast Error vs. Horizon')
plt.grid(True)
plt.show()

In [ ]:
# Plot the results
plt.figure(figsize=(14, 8))

# Plot historical data and EnKF estimate
plt.plot(data.index, prices, 'k-', label='True Price')
plt.plot(data.index, x_mean_hist, 'r--', label='EnKF Estimate')

# Plot forecasts
last_date = data.index[-1]
for horizon, forecast_values in forecasts.items():
    forecast_dates = pd.to_datetime([last_date + pd.DateOffset(days=i+1) for i in range(horizon)])
    plt.plot(forecast_dates, forecast_values, 'o-', label=f'{horizon}-day Forecast')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title(f'EnKF State Estimation and Forecast for {ticker}')
plt.legend()
plt.grid(True)
plt.show()